## Question 15.2
### Part 1

To begin, we import the libraries to use and create a `pandas` `DataFrame` for the data to be used in this problem.

In [1]:
import numpy as np
import pandas as pd
from pulp import *
import os

path_data = os.path.join("..", "data-raw", "diet.xls")

In [2]:
df_raw = pd.read_excel(path_data)

# Removing the rows with the min max constraints.
df = df_raw[np.isfinite(df_raw["Price/ Serving"])]

# Experimenting with renaming columns.
# df.columns = df.columns.str.strip().str.lower().str.replace("/ ", "per").str.replace(" ", "_")

# Viewing the data
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [3]:
df.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


Now I create `list` for the foods, and a `dict` for each column in the data with a constraint to be met for the optimization problem. (Admittedly, this is not the most elegant code. Also, note that these values could be imported directly from the last several rows from the .xls file with the rest of the data; nonetheless, I choose to simply "hard-code" the values here.)

In [4]:
# Peeking at the constraints to hardcode.
constr = df_raw[-2:].drop(["Foods", "Price/ Serving",
                           "Serving Size"], axis="columns")
constr

,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [5]:
food = df["Foods"].tolist()

# Note that it's faster to convert with Series() and to_dict()
# Reference: https://stackoverflow.com/questions/17426292/what-is-the-most-efficient-way-to-create-a-dictionary-of-two-pandas-dataframe-co/34004356
# cost = dict(zip(df["Foods"], df["Price/ Serving"]))
cost = pd.Series(df["Price/ Serving"].values, index=df["Foods"]).to_dict()
cal = pd.Series(df["Calories"].values, index=df["Foods"]).to_dict()
chol = pd.Series(df["Cholesterol mg"].values, index=df["Foods"]).to_dict()
fat = pd.Series(df["Total_Fat g"].values, index=df["Foods"]).to_dict()
sodium = pd.Series(df["Sodium mg"].values, index=df["Foods"]).to_dict()
carb = pd.Series(df["Carbohydrates g"].values, index=df["Foods"]).to_dict()
fiber = pd.Series(df["Dietary_Fiber g"].values, index=df["Foods"]).to_dict()
protein = pd.Series(df["Protein g"].values, index=df["Foods"]).to_dict()
vita = pd.Series(df["Vit_A IU"].values, index=df["Foods"]).to_dict()
vitc = pd.Series(df["Vit_C IU"].values, index=df["Foods"]).to_dict()
calcium = pd.Series(df["Calcium mg"].values, index=df["Foods"]).to_dict()
iron = pd.Series(df["Iron mg"].values, index=df["Foods"]).to_dict()

In [6]:
# Reviewing what one of the `dict`s looks like.
# Reference: https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
{k: cost[k] for k in list(cost)[:5]}

{'Apple Pie': 0.16,
 'Beanbacn Soup,W/Watr': 0.67,
 'Kiwifruit,Raw,Fresh': 0.49,
 'Popcorn,Air-Popped': 0.04,
 'Pretzels': 0.12}

Next, I setup the problem and use Pulp's `solve()` function to do the optimization. As noted by the TA, this problem is very similar to the "blender" problem in the PulP documentation, hence there is not much to explain. (See https://pythonhosted.org/PuLP/CaseStudies/a_blending_problem.html#full-formulation.)

In [7]:
# Defining the "problem".
prob1 = LpProblem("part 1", LpMinimize)

In [8]:
# Creating a `dict` to store the reference variables.
food_vars = LpVariable.dicts("food", food, 0)

In [9]:
# Defining the objective function.
prob1 += lpSum([cost[i] * food_vars[i]
                for i in food]), "total cost of food per person"

In [10]:
# The constraints for calories and each macro/micro-nutrient.
prob1 += lpSum([cal[i] * food_vars[i] for i in food]) >= 1500, "min cal"
prob1 += lpSum([cal[i] * food_vars[i] for i in food]) <= 2500, "max cal"
prob1 += lpSum([chol[i] * food_vars[i] for i in food]) >= 30, "min chol"
prob1 += lpSum([chol[i] * food_vars[i] for i in food]) <= 240, "max chol"
prob1 += lpSum([fat[i] * food_vars[i] for i in food]) >= 20, "min fat"
prob1 += lpSum([fat[i] * food_vars[i] for i in food]) <= 70, "max fat"
prob1 += lpSum([sodium[i] * food_vars[i] for i in food]) >= 800, "min sodium"
prob1 += lpSum([sodium[i] * food_vars[i] for i in food]) <= 2000, "max sodium"
prob1 += lpSum([carb[i] * food_vars[i] for i in food]) >= 130, "min carb"
prob1 += lpSum([carb[i] * food_vars[i] for i in food]) <= 450, "max carb"
prob1 += lpSum([fiber[i] * food_vars[i] for i in food]) >= 125, "min fiber"
prob1 += lpSum([fiber[i] * food_vars[i] for i in food]) <= 250, "max fiber"
prob1 += lpSum([protein[i] * food_vars[i] for i in food]) >= 60, "min protein"
prob1 += lpSum([protein[i] * food_vars[i] for i in food]) <= 100, "max protein"
prob1 += lpSum([vita[i] * food_vars[i] for i in food]) >= 1000, "min vita"
prob1 += lpSum([vita[i] * food_vars[i] for i in food]) <= 10000, "max vita"
prob1 += lpSum([vitc[i] * food_vars[i] for i in food]) >= 400, "min vitc"
prob1 += lpSum([vitc[i] * food_vars[i] for i in food]) <= 5000, "max vitc"
prob1 += lpSum([calcium[i] * food_vars[i] for i in food]) >= 700, "min calcium"
prob1 += lpSum([calcium[i] * food_vars[i]
                for i in food]) <= 1500, "max calcium"
prob1 += lpSum([iron[i] * food_vars[i] for i in food]) >= 10, "min iron"
prob1 += lpSum([iron[i] * food_vars[i] for i in food]) <= 40, "max iron"

In [11]:
# Solving.
prob1.solve()

1

In [12]:
# Interpreting the status code.
print("status:", LpStatus[prob1.status])

status: Optimal


In [13]:
# Reviewing the variables with non-zero optimum values.
for var in prob1.variables():
    if var.varValue > 0:
        # print(str(var.varValue) + " units of " + str(var).replace("food_", ""))
        # Or...
        print("%.2f units of %s" %
              (var.varValue, str(var).replace("food_", "")))

52.64 units of Celery,_Raw
0.26 units of Frozen_Broccoli
63.99 units of Lettuce,Iceberg,Raw
2.29 units of Oranges
0.14 units of Poached_Eggs
13.87 units of Popcorn,Air_Popped


In [14]:
# Reviewing the optimized objective function value.
print("total cost per person = $%.2f" % value(prob1.objective))

total cost per person = $4.34


It looks like I calculate the values that I should find, per the homework instructions.

### Part 2

This part adds a couple of constraints to the part 1 problem. Notably, a set of binary variables are needed to indicate whether or not a food is chosen (as suggested by the hint).

A couple of notes about the implementation:
+ A maximum serving size of `1000000` is used, even though the instructions don't explicity call for this. (Setting a maximum is simply good practice when also setting a minimum. Additionally, note that optimization solver doesn't "understand" the problem beyond what the user specifies, so it can calculate unreasonable values (e.g. a negative serving size) if minimums/maximums are not properly specified.)
+ The choice of foods to include for criteria "c" (regarding meat/poultry/fish/eggs) is a bit ambiguous, as noted by the homework instructions. I choose NOT to include foods involving soups.
    
Aside from these caveats, the implementation is very similar to that of part 1, so my comments here are focused on comparisons with the previous set-up.

In [15]:
# The same libraries and data are needed (so there is no need to re-import them).
# Skip straight to describing the problem.
prob2 = LpProblem("part 2", LpMinimize)

In [16]:
# The same dict of reference values as before.
food_vars = LpVariable.dicts("food", food, 0)

In [17]:
# A new set of variables used to indicate whether or not a food is chosen.
choice_vars = LpVariable.dicts("choice", food, 0, 1, "Binary")
# Or...
# choice_vars = LpVariable.dicts("choice", food, 0, 1, cat=LpInteger)

In [18]:
# The same objective function as before.
prob2 += lpSum([cost[i] * food_vars[i]
                for i in food]), "total cost of food per person"

In [19]:
# The same 22 constraints as before, hard-coded again.
prob2 += lpSum([cal[i] * food_vars[i] for i in food]) >= 1500, "min cal"
prob2 += lpSum([cal[i] * food_vars[i] for i in food]) <= 2500, "max cal"
prob2 += lpSum([chol[i] * food_vars[i] for i in food]) >= 30, "min chol"
prob2 += lpSum([chol[i] * food_vars[i] for i in food]) <= 240, "max chol"
prob2 += lpSum([fat[i] * food_vars[i] for i in food]) >= 20, "min fat"
prob2 += lpSum([fat[i] * food_vars[i] for i in food]) <= 70, "max fat"
prob2 += lpSum([sodium[i] * food_vars[i] for i in food]) >= 800, "min sodium"
prob2 += lpSum([sodium[i] * food_vars[i] for i in food]) <= 2000, "max sodium"
prob2 += lpSum([carb[i] * food_vars[i] for i in food]) >= 130, "min carb"
prob2 += lpSum([carb[i] * food_vars[i] for i in food]) <= 450, "max carb"
prob2 += lpSum([fiber[i] * food_vars[i] for i in food]) >= 125, "min fiber"
prob2 += lpSum([fiber[i] * food_vars[i] for i in food]) <= 250, "max fiber"
prob2 += lpSum([protein[i] * food_vars[i] for i in food]) >= 60, "min protein"
prob2 += lpSum([protein[i] * food_vars[i] for i in food]) <= 100, "max protein"
prob2 += lpSum([vita[i] * food_vars[i] for i in food]) >= 1000, "min vita"
prob2 += lpSum([vita[i] * food_vars[i] for i in food]) <= 10000, "max vita"
prob2 += lpSum([vitc[i] * food_vars[i] for i in food]) >= 400, "min vitc"
prob2 += lpSum([vitc[i] * food_vars[i] for i in food]) <= 5000, "max vitc"
prob2 += lpSum([calcium[i] * food_vars[i] for i in food]) >= 700, "min calcium"
prob2 += lpSum([calcium[i] * food_vars[i]
                for i in food]) <= 1500, "max calcium"
prob2 += lpSum([iron[i] * food_vars[i] for i in food]) >= 10, "min iron"
prob2 += lpSum([iron[i] * food_vars[i] for i in food]) <= 40, "max iron"

In [20]:
# a constraints
for f in food:
    prob2 += food_vars[f] <= 1000000 * \
        choice_vars[f], "max serving size, %s" % f

for f in food:
    prob2 += food_vars[f] >= 0.1 * choice_vars[f], "min serving size, %s" % f

In [21]:
# b constraint
# Limit these foods to one or none
prob2 += choice_vars["Frozen Broccoli"] + choice_vars["Celery, Raw"] <= 1, "b"

In [22]:
# Reviewing the different foods to see what may be classified as meat/poultry/fish/eggs.
# food

In [23]:
# c constraint
prob2 += (choice_vars["Roasted Chicken"] +
          choice_vars["Poached Eggs"] +
          choice_vars["Scrambled Eggs"] +
          choice_vars["Bologna,Turkey"] +
          choice_vars["Frankfurter, Beef"] +
          choice_vars["Ham,Sliced,Extralean"] +
          choice_vars["Kielbasa,Prk"] +
          choice_vars["Pizza W/Pepperoni"] +
          choice_vars["Hamburger W/Toppings"] +
          choice_vars["Hotdog, Plain"] +
          choice_vars["Pork"] +
          choice_vars["Sardines in Oil"] +
          # choice_vars["White Tuna in Water"] +
          # choice_vars["Chicknoodl Soup"] +
          # choice_vars["Splt Pea&Hamsoup"] +
          # choice_vars["Vegetbeef Soup"] +
          # choice_vars["Beanbacn Soup,W/Watr"]) >= 3, "c"
          choice_vars["White Tuna in Water"]) >= 3, "c"

In [24]:
prob2.solve()

1

In [25]:
# print("status:", LpStatus[prob2.status])

In [26]:
for var in prob2.variables():
    # Ignore the indicator variables.
    if var.varValue > 0 and var.varValue != 1:
        print("%.2f units of %s" %
              (var.varValue, str(var).replace("food_", "")))

42.40 units of Celery,_Raw
0.10 units of Kielbasa,Prk
82.80 units of Lettuce,Iceberg,Raw
3.08 units of Oranges
1.94 units of Peanut_Butter
0.10 units of Poached_Eggs
13.22 units of Popcorn,Air_Popped
0.10 units of Scrambled_Eggs


Note that 1 of celery and broccoli is chosen, and that at least 3 meat/poultry/fish/eggs items are selected (`Kielbasa,Prk`, `Poached_Eggs`, and `Scrambled_Eggs`), according to our specifications. Additionally, we can verify that each of the final items have a serving size of at least 0.1.

In [27]:
print("total cost per person = $%.2f" % value(prob2.objective))

total cost per person = $4.51


Note that the objective function value (i.e. total cost) is slightly higher here than that found in part 1. This is to be expected when additional constraints are added.

### Part 2, Large Data Set

Now, as a final exercise, let's (try to) re-implement the part 2 set-up for the large data set. The major difference with this set-up is that minimizing cholesterol (not cost) is the goal. (In fact, cost is not even provided with the data set.)

Additionally, in terms of implementation, I do not hard-code the maximum and minimum values here. Instead, I create dictionaries for each set of limits and iterate over them when calling `lpSum()`
in the problem set-up.

In [28]:
path_data_large = os.path.join("..", "data-raw", "diet_largeSummer2018.xls")

In [29]:
df_large_raw = pd.read_excel(path_data_large, skiprows=1, header=0)

In [30]:
# Removing the constraint rows.
df_large = df_large_raw[:-4]

# Reviewing what the data looks like.
df_large.head()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,"Butter, salted",0.85,0.06,717,15.87,3000.0,24,0.02,2,24,...,0.034,0.042,0.11,0.003,3,0.17,7,215.0,NaN,51.368
1,"Butter, whipped, with salt",0.85,0.06,717,15.87,2999.0,24,0.16,2,23,...,0.034,0.042,0.11,0.003,3,0.13,7,219.0,NaN,50.489
2,"Butter oil, anhydrous",0.28,0,876,0.24,3665.0,4,0,0,3,...,0.005,0.003,0.01,0.001,0,0.01,8.6,256.0,NaN,61.924
3,"Cheese, blue",21.4,2.34,353,42.41,1477.0,528,0.31,23,387,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75.0,NaN,18.669
4,"Cheese, brick",23.24,2.79,371,41.11,1552.0,674,0.43,24,451,...,0.351,0.118,0.288,0.065,20,1.26,2.5,94.0,NaN,18.764


In [31]:
# df_large.tail()

In [32]:
# Note that there are NA values that need to be removed in order for the optimization to work.
df_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7146 entries, 0 to 7145
Data columns (total 31 columns):
Long_Desc                         7146 non-null object
Protein                           7146 non-null object
Carbohydrate, by difference       7146 non-null object
Energy                            7146 non-null object
Water                             7142 non-null object
Energy.1                          7146 non-null float64
Calcium, Ca                       6983 non-null object
Iron, Fe                          7016 non-null object
Magnesium, Mg                     6632 non-null object
Phosphorus, P                     6733 non-null object
Potassium, K                      6788 non-null object
Sodium, Na                        7063 non-null object
Zinc, Zn                          6607 non-null object
Copper, Cu                        6545 non-null object
Manganese, Mn                     5732 non-null object
Selenium, Se                      5882 non-null object
Vitamin A, R

In [33]:
# Reviewing how many NAs there are.
df_large.isnull().sum().sum()

31962

In [34]:
df_large.fillna(value=0.0, inplace=True)

# Checking that the NAs were corrected.
# df_large.isnull().sum().sum()

C:\Users\aelhabr\Anaconda3\lib\site-packages\pandas\core\frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [35]:
# Subsetting to create a constraints `DataFrame`.
constr = df_large_raw[-3:]
constr = constr.iloc[[0, 2], ].drop(
    ["Long_Desc", "Cholesterol", "Fatty acids, total trans", "Fatty acids, total saturated"], axis="columns")
# constr

In [36]:
# Coercing the constraints to `dict`s.
constr_min = constr.to_dict(orient="records")[0]
constr_max = constr.to_dict(orient="records")[1]

# Reviewing one set of constraint values.
constr_max

{'Calcium, Ca': 2500,
 'Carbohydrate, by difference': 1000000,
 'Copper, Cu': 10,
 'Energy': 1000000,
 'Energy.1': 1000000.0,
 'Folate, total': 1000,
 'Iron, Fe': 45,
 'Magnesium, Mg': 400,
 'Manganese, Mn': 11,
 'Niacin': 35,
 'Pantothenic acid': 1000000,
 'Phosphorus, P': 4000,
 'Potassium, K': 1000000,
 'Protein': 1000000,
 'Riboflavin': 1000000,
 'Selenium, Se': 400,
 'Sodium, Na': 2300,
 'Thiamin': 1000000,
 'Vitamin A, RAE': 3000,
 'Vitamin B-12': 1000000,
 'Vitamin B-6': 100,
 'Vitamin C, total ascorbic acid': 2000,
 'Vitamin D': 2000,
 'Vitamin E (alpha-tocopherol)': 1000,
 'Vitamin K (phylloquinone)': 1000000,
 'Water': 1000000,
 'Zinc, Zn': 40}

In [37]:
# Creating the `list`/`dict` for the reference variables.
food_large = df_large["Long_Desc"].tolist()
chol_large = pd.Series(
    df_large["Cholesterol"].values, index=df_large["Long_Desc"]).to_dict()

In [38]:
# Reviewing the reference variables.
food_large[:5]

['Butter, salted',
 'Butter, whipped, with salt',
 'Butter oil, anhydrous',
 'Cheese, blue',
 'Cheese, brick']

In [39]:
{k: chol_large[k] for k in list(chol_large)[:5]}

{'Carbonated beverage, SPRITE, lemon-lime, without caffeine': 0.0,
 'Cereals, QUAKER, corn grits, instant, butter flavor, dry': 1.0,
 'Lamb, domestic, leg, sirloin half, separable lean only, trimmed': 92.0,
 'Milk, buttermilk, dried': 69.0,
 'Soup, pea, green, mix, dehydrated, dry form': 1.0}

In [40]:
# Setting up the optimization, in a similar manner as before.
prob2_large = LpProblem("part 2 large", LpMinimize)
food_vars_large = LpVariable.dicts("food", food_large, 0)
prob2_large += lpSum([chol_large[i] * food_vars_large[i]
                      for i in food_large]), "total cholesterol of food per person"

In [41]:
# Reviewing the columns that have constraints.
# df_large.columns[1:28]

In [42]:
# Creating the maximum/minimum limits. This takes a while to complete...
for col in df_large.columns[1:28]:
    # print(col)
    # print(df_large[col].values[:5])
    col_vals = pd.Series(df_large[col].values,
                         index=df_large["Long_Desc"]).to_dict()
    # col_vals.head()
    constr_min_i = constr_min[col]
    constr_max_i = constr_max[col]
    prob2_large += lpSum([col_vals[i] * food_vars_large[i]
                          for i in food_large]) >= constr_min_i, ""
    prob2_large += lpSum([col_vals[i] * food_vars_large[i]
                          for i in food_large]) <= constr_max_i, ""

In [43]:
# Write it to a file for quick-reference later.
# prob2_large.writeLP("prob2_large.lp")

In [44]:
prob2_large.solve()

1

In [45]:
for var in prob2_large.variables():
    # Ignore the indicator variables.
    if var.varValue > 0 and var.varValue != 1:
        print("%.2f units of %s" %
              (var.varValue, str(var).replace("food_", "")))

0.06 units of Beans,_adzuki,_mature_seeds,_raw
0.07 units of Broccoli_raab,_raw
0.43 units of Cocoa_mix,_no_sugar_added,_powder
0.15 units of Egg,_white,_dried,_flakes,_glucose_reduced
0.74 units of Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_NUTRAMIGEN,_with_iron,_p
0.43 units of Infant_formula,_NESTLE,_GOOD_START_ESSENTIALS__SOY,__with_iron,
0.05 units of Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu
0.15 units of Margarine_like_spread,_approximately_60%_fat,_tub,_soybean_(hyd
0.26 units of Mung_beans,_mature_seeds,_raw
0.18 units of Nuts,_mixed_nuts,_dry_roasted,_with_peanuts,_with_salt_added
1.18 units of Oil,_vegetable,_sunflower,_linoleic,_(hydrogenated)
0.10 units of Seeds,_sunflower_seed_kernels,_dry_roasted,_with_salt_added
0.03 units of Snacks,_potato_chips,_fat_free,_made_with_olestra
0.07 units of Spices,_paprika
0.55 units of Tomatoes,_sun_dried
9999.69 units of Water,_bottled,_non_carbonated,_CALISTOGA


In [46]:
print("total cost = $%.2f" % value(prob2_large.objective))

total cost = $0.00


Unfortunately, although I get an "optimal" solution, it doesn't really appear to make sense--the total cost is `$0`. This seemingly erroneous solution may be due to the "non-convexity" of this problem. (The homework instructions note that there is not a single right answer to this problem.) Nonetheless, I believe my implementation is correct (or nearly correct).